In [21]:
import pandas as pd
from pprint import pprint
from collections import Counter
df_tennis = pd.read_csv('PlayTennis.csv')
#df_tennis=df_tennis[:11] #for checking accuracy
# print(df_tennis)
group = df_tennis.groupby('Outlook')
print(group.groups)

# for name, g in group:
#   print(name)
#   print(g)

# print(len(df_tennis.index))
count = Counter([x for x in df_tennis['Outlook']])
print(count)
print(max(count, key = lambda x : count[x]))
a = lambda x : count[x]
print(a('Sunny'))

for attrval, df_tennis in df_tennis.groupby('Outlook'):
  print('attr', attrval)
  print(df_tennis)

{'Overcast': [2, 6, 11, 12], 'Rain': [3, 4, 5, 9, 13], 'Sunny': [0, 1, 7, 8, 10]}
Counter({'Sunny': 5, 'Rain': 5, 'Overcast': 4})
Sunny
5
attr Overcast
   PlayTennis   Outlook Temperature Humidity    Wind
2         Yes  Overcast         Hot     High    Weak
6         Yes  Overcast        Cool   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
attr Rain
   PlayTennis Outlook Temperature Humidity    Wind
3         Yes    Rain        Mild     High    Weak
4         Yes    Rain        Cool   Normal    Weak
5          No    Rain        Cool   Normal  Strong
9         Yes    Rain        Mild   Normal    Weak
13         No    Rain        Mild     High  Strong
attr Sunny
   PlayTennis Outlook Temperature Humidity    Wind
0          No   Sunny         Hot     High    Weak
1          No   Sunny         Hot     High  Strong
7          No   Sunny        Mild     High    Weak
8         Yes   Sunny        Cool   Normal    Weak
1

In [22]:
#Function to calculate the entropy of collection S
import math

def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs])

def entropy_of_list(ls):
    print('\n-----------------------------------------------------------------------------------------------------------\n',ls)
    class_count = Counter(x for x in ls)# Counter calculates yes or no of an attr value eg; no. of yes in outlook = overcast
    print('Target attribute class count(Yes/No)=',dict(class_count))
    print([x for x in ls])
    total_instances = len(ls)  #total no. of outlook = overcast
    probs = [x / total_instances for x in class_count.values()] #there will be 2 keys in the class count - yes and no  
    print('probablity',probs)
    return entropy(probs) # Call Entropy

In [23]:
def information_gain(df, split_attr,tg_attr):
    df_split = df.groupby(split_attr) # group the data based on attribute values
    print(f'\n\n\nAttribute-{split_attr}')
    pprint(df_split.groups)
    datasetlen = len(df.index) #calc len of dataset
    print('\ntarget attribute', tg_attr)
    for key, item in df_split:
        print(key,'\n\n', df_split.get_group(key), "--------------------------------------------------------------\n\n")
    
    
    df_agg = df_split.agg({tg_attr:[entropy_of_list, lambda x: len(x)/datasetlen]})[tg_attr] # only applying the function to tg_attr column (PlayTennis) i.e Yes or NO
    #collecting both entropy and propotion of each attr value
    # eg - outlook = overcast entropy = 0.2, prop = 0.3 ;; outlook = sunny entropy = 0.7, prop = 0.13 etc
    #on the target attr col applying the entropy of list func & also calc fraction
    # since it is grouped data - the functions are applied separately to each group
   
    df_agg.columns=['Entropy','Proportion']
    print('\n*****aggregate****\n',df_agg,'\n\n\n')
    # Calculate Information Gain:
    new_entropy = sum( df_agg['Entropy'] * df_agg['Proportion'])
    print('old entropy', df[tg_attr])
    old_entropy = entropy_of_list(df[tg_attr]) #H(S) i.e entropy of Yes & No of entire table
    return old_entropy - new_entropy

In [24]:
#default attribute - S is the initial attribute 
def id3(df, tg_attr, attr, default_class=None,default_attr='S'):
    

    pos_neg = Counter(x for x in df[tg_attr])# class of YES /NO
    print('Positive Negative',pos_neg,len(pos_neg),max(pos_neg.keys()),'\n')
    if len(pos_neg) == 1: # if there is only Yes or only No, return Yes or No.
        return next(iter(pos_neg))  
    
    elif df.empty or (not attr):
        return default_class  # Return None for Empty Data Set or when no attributes are present
    
    else:
        #whatever is more i.e Yes or No 
        default_class = max(pos_neg.keys()) #No of YES and NO Class -> this is of no use btw 
        # print(f'default class {default_class}')
        gains={}
        # print('On attribute ', default_attr, attr)

        #calc info gain of each attribute 
        for a in attr:
            gains[a] = information_gain(df, a, tg_attr)
            print(f'Information gain of {a}:{gains[a]:0.4f}')
        
        #best attribute   
        best_attr = max(gains, key=lambda x: gains[x]) # iterating through the dict keys, and getting the values
       
        tree = {best_attr:{}} # Initiate the tree with best attribute as a node  
        attr.remove(best_attr) #remove the best attribute
      
        for attr_val, data in df.groupby(best_attr):
            subtree = id3(data,tg_attr, attr,default_class,best_attr)
            tree[best_attr][attr_val] = subtree
            # print('best_attr, attr_val',best_attr, attr_val)
            # print('Tree',tree)
        return tree

In [25]:
# Get Predictor Names (all but 'class')
attr = list(df_tennis.columns)
# print("List of Attributes:", attr) 
attr.remove('PlayTennis') #Remove the class attribute 
print("Predicting Attributes:", attr)

Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [26]:
tree = id3(df_tennis,'PlayTennis',attr)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
ba = next(iter(tree))
# print("Best Attribute :\n",ba)
# print("Tree Keys:\n",tree[ba].keys())

Positive Negative Counter({'No': 3, 'Yes': 2}) 2 Yes 




Attribute-Outlook
{'Sunny': [0, 1, 7, 8, 10]}

target attribute PlayTennis
Sunny 

    PlayTennis Outlook Temperature Humidity    Wind
0          No   Sunny         Hot     High    Weak
1          No   Sunny         Hot     High  Strong
7          No   Sunny        Mild     High    Weak
8         Yes   Sunny        Cool   Normal    Weak
10        Yes   Sunny        Mild   Normal  Strong --------------------------------------------------------------



-----------------------------------------------------------------------------------------------------------
 0      No
1      No
7      No
8     Yes
10    Yes
Name: PlayTennis, dtype: object
Target attribute class count(Yes/No)= {'No': 3, 'Yes': 2}
['No', 'No', 'No', 'Yes', 'Yes']
probablity [0.6, 0.4]

*****aggregate****
           Entropy  Proportion
Outlook                      
Sunny    0.970951         1.0 



old entropy 0      No
1      No
7      No
8     Yes
10    Yes
Name:

In [27]:
def classify(instance, tree,default=None): # Instance of Play Tennis with Predicted  
    attribute = next(iter(tree)) # Outlook/Humidity/Wind     
    if instance[attribute] in tree[attribute].keys(): # Value of the attributs in  set of Tree keys  
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): # this is a tree, delve deeper
            return classify(instance, result)
        else:
            return result # this is a single attribute (not a dictionary -- Yes or No)
    else:
        return default

In [28]:

df_new=pd.read_csv('PlayTennis.csv')
df_new=df_new[11:]
df_new['predicted'] = df_new.apply(classify, axis=1, args=(tree,'?')) 
print(df_new)

   PlayTennis   Outlook Temperature Humidity    Wind predicted
11        Yes  Overcast        Mild     High  Strong        No
12        Yes  Overcast         Hot   Normal    Weak       Yes
13         No      Rain        Mild     High  Strong        No


<h2>Just for understanding</h2>

In [29]:
def displaylist(ls):
  print(ls)
  return 0

In [30]:
#********************************Just for understanding**********************************
from collections import Counter
cnt = Counter(x for x in df_tennis['Humidity'])# class of YES /NO
print(cnt)
print(max(cnt.keys()))
#iter - returns an iterator object and next elements can be accessed using next()
# lst ={'foo','bar','baz'}
# print(next(iter(lst)))

# print(sum([1,2,3,4]))

# dic = {'abc':4}
# print(len(dic))

group = df_tennis.groupby('Wind')
for key, item in group:
    print(group.get_group(key), "\n\n")

df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [20,30,40]],
                  columns=['A', 'B', 'C'])
print(df,'\n')
# dfa = df['A'] #returns a dataframe again
# print(dfa)
nobs=1
# print(df.agg(['sum', 'min']),'\n') #same function applied to each column
# print(df.agg({'A' : ['sum', lambda x: len(x)/nobs], 'B' : ['min', 'max']})) #different function applied to different columns
print(group.agg({'Wind' : ['sum', lambda x: len(x)/nobs]})) #different function applied to different columns 

Counter({'High': 3, 'Normal': 2})
Normal
   PlayTennis Outlook Temperature Humidity    Wind
1          No   Sunny         Hot     High  Strong
10        Yes   Sunny        Mild   Normal  Strong 


  PlayTennis Outlook Temperature Humidity  Wind
0         No   Sunny         Hot     High  Weak
7         No   Sunny        Mild     High  Weak
8        Yes   Sunny        Cool   Normal  Weak 


    A   B   C
0   1   2   3
1   4   5   6
2   7   8   9
3  20  30  40 

                Wind           
                 sum <lambda_0>
Wind                           
Strong  StrongStrong        2.0
Weak    WeakWeakWeak        3.0
